In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score

/Users/satsaxen/anaconda/lib/python2.7/site-packages/pandas/computation/__init__.py:19: UserWarning: The installed version of numexpr 2.4.4 is not supported in pandas and will be not be used

  UserWarning)


In [2]:
df_train=pd.read_csv('act_train.csv')
df_test=pd.read_csv('act_test.csv')

In [3]:
df_train.shape

(2197291, 15)

In [4]:
df_train.isnull().sum()

people_id                  0
activity_id                0
date                       0
activity_category          0
char_1               2039676
char_2               2039676
char_3               2039676
char_4               2039676
char_5               2039676
char_6               2039676
char_7               2039676
char_8               2039676
char_9               2039676
char_10               157615
outcome                    0
dtype: int64

In [5]:
df_train.head()

,people_id,activity_id,date,activity_category,char_1,char_2,char_3,char_4,char_5,char_6,char_7,char_8,char_9,char_10,outcome
0,ppl_100,act2_1734928,2023-08-26,type 4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,type 76,0
1,ppl_100,act2_2434093,2022-09-27,type 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,type 1,0
2,ppl_100,act2_3404049,2022-09-27,type 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,type 1,0
3,ppl_100,act2_3651215,2023-08-04,type 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,type 1,0
4,ppl_100,act2_4109017,2023-08-26,type 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,type 1,0


In [6]:
probs = {}
print('Probabilities for each activity category:')
for a in df_train['activity_category'].unique():
    probs[a] = df_train.loc[df_train.activity_category == a]['outcome'].mean()
    print(a, probs[a])

Probabilities for each activity category:
('type 4', 0.48920540814113223)
('type 2', 0.51032350558151307)
('type 3', 0.25198878455920709)
('type 5', 0.48024291332966518)
('type 1', 0.41132506423880977)
('type 7', 0.39974659486854608)
('type 6', 0.55584293439924759)


In [7]:
preds_train = [probs[a] for a in df_train['activity_category']]
preds_test = [probs[a] for a in df_test['activity_category']]

In [8]:
print('Local AUC: ' + str(roc_auc_score(df_train['outcome'], preds_train)))

sub = pd.DataFrame()
sub['activity_id'] = df_test['activity_id']
sub['outcome'] = preds_test
sub.to_csv('beat_the_benchmark.csv', index=False)

Local AUC: 0.593276763981


In [9]:
person = pd.read_csv("people.csv")
person.head()

,people_id,char_1,group_1,char_2,date,char_3,char_4,char_5,char_6,char_7,...,char_29,char_30,char_31,char_32,char_33,char_34,char_35,char_36,char_37,char_38
0,ppl_100,type 2,group 17304,type 2,2021-06-29,type 5,type 5,type 5,type 3,type 11,...,False,True,True,False,False,True,True,True,False,36
1,ppl_100002,type 2,group 8688,type 3,2021-01-06,type 28,type 9,type 5,type 3,type 11,...,False,True,True,True,True,True,True,True,False,76
2,ppl_100003,type 2,group 33592,type 3,2022-06-10,type 4,type 8,type 5,type 2,type 5,...,False,False,True,True,True,True,False,True,True,99
3,ppl_100004,type 2,group 22593,type 3,2022-07-20,type 40,type 25,type 9,type 4,type 16,...,True,True,True,True,True,True,True,True,True,76
4,ppl_100006,type 2,group 6534,type 3,2022-07-27,type 40,type 25,type 9,type 3,type 8,...,False,False,True,False,False,False,True,True,False,84


In [10]:
person.isnull().sum()

people_id    0
char_1       0
group_1      0
char_2       0
date         0
char_3       0
char_4       0
char_5       0
char_6       0
char_7       0
char_8       0
char_9       0
char_10      0
char_11      0
char_12      0
char_13      0
char_14      0
char_15      0
char_16      0
char_17      0
char_18      0
char_19      0
char_20      0
char_21      0
char_22      0
char_23      0
char_24      0
char_25      0
char_26      0
char_27      0
char_28      0
char_29      0
char_30      0
char_31      0
char_32      0
char_33      0
char_34      0
char_35      0
char_36      0
char_37      0
char_38      0
dtype: int64

In [11]:
import datetime
import pandas as pd
import numpy as np
from sklearn.cross_validation import KFold
from sklearn.cross_validation import train_test_split
from sklearn.metrics import roc_auc_score
import xgboost as xgb
import random
from operator import itemgetter
import time
import copy

random.seed(2016)

In [12]:
X = pd.merge(df_train, person, on='people_id')

In [13]:
X.head()

,people_id,activity_id,date_x,activity_category,char_1_x,char_2_x,char_3_x,char_4_x,char_5_x,char_6_x,...,char_29,char_30,char_31,char_32,char_33,char_34,char_35,char_36,char_37,char_38
0,ppl_100,act2_1734928,2023-08-26,type 4,NaN,NaN,NaN,NaN,NaN,NaN,...,False,True,True,False,False,True,True,True,False,36
1,ppl_100,act2_2434093,2022-09-27,type 2,NaN,NaN,NaN,NaN,NaN,NaN,...,False,True,True,False,False,True,True,True,False,36
2,ppl_100,act2_3404049,2022-09-27,type 2,NaN,NaN,NaN,NaN,NaN,NaN,...,False,True,True,False,False,True,True,True,False,36
3,ppl_100,act2_3651215,2023-08-04,type 2,NaN,NaN,NaN,NaN,NaN,NaN,...,False,True,True,False,False,True,True,True,False,36
4,ppl_100,act2_4109017,2023-08-26,type 2,NaN,NaN,NaN,NaN,NaN,NaN,...,False,True,True,False,False,True,True,True,False,36


In [14]:
X.shape

(2197291, 55)

In [15]:
X.isnull().sum()

people_id                  0
activity_id                0
date_x                     0
activity_category          0
char_1_x             2039676
char_2_x             2039676
char_3_x             2039676
char_4_x             2039676
char_5_x             2039676
char_6_x             2039676
char_7_x             2039676
char_8_x             2039676
char_9_x             2039676
char_10_x             157615
outcome                    0
char_1_y                   0
group_1                    0
char_2_y                   0
date_y                     0
char_3_y                   0
char_4_y                   0
char_5_y                   0
char_6_y                   0
char_7_y                   0
char_8_y                   0
char_9_y                   0
char_10_y                  0
char_11                    0
char_12                    0
char_13                    0
char_14                    0
char_15                    0
char_16                    0
char_17                    0
char_18       

# Data Preprocessing - Missing value imputation

In [16]:
import pandas as pd
import numpy as np

from sklearn.base import TransformerMixin

class DataFrameImputer(TransformerMixin):

    def __init__(self):
        """Impute missing values.

        Columns of dtype object are imputed with the most frequent value 
        in column.

        Columns of other types are imputed with mean of column.

        """
    def fit(self, X, y=None):

        self.fill = pd.Series([X[c].value_counts().index[0]
            if X[c].dtype == np.dtype('O') else X[c].mean() for c in X],
            index=X.columns)

        return self

    def transform(self, X, y=None):
        return X.fillna(self.fill)

In [17]:
X = DataFrameImputer().fit_transform(X)

In [18]:
X.head()

,people_id,activity_id,date_x,activity_category,char_1_x,char_2_x,char_3_x,char_4_x,char_5_x,char_6_x,...,char_29,char_30,char_31,char_32,char_33,char_34,char_35,char_36,char_37,char_38
0,ppl_100,act2_1734928,2023-08-26,type 4,type 2,type 2,type 1,type 3,type 6,type 2,...,False,True,True,False,False,True,True,True,False,36
1,ppl_100,act2_2434093,2022-09-27,type 2,type 2,type 2,type 1,type 3,type 6,type 2,...,False,True,True,False,False,True,True,True,False,36
2,ppl_100,act2_3404049,2022-09-27,type 2,type 2,type 2,type 1,type 3,type 6,type 2,...,False,True,True,False,False,True,True,True,False,36
3,ppl_100,act2_3651215,2023-08-04,type 2,type 2,type 2,type 1,type 3,type 6,type 2,...,False,True,True,False,False,True,True,True,False,36
4,ppl_100,act2_4109017,2023-08-26,type 2,type 2,type 2,type 1,type 3,type 6,type 2,...,False,True,True,False,False,True,True,True,False,36


# Converting Boolean Features to 0 and 1. 

In [19]:
# Convert booleans to integers.
p_logi = X.select_dtypes(include=['bool']).columns
X[p_logi] = X[p_logi].astype('int')
del p_logi

X['date_x'] = pd.to_datetime(X['date_x'])
X['date_y'] = pd.to_datetime(X['date_y'])

# Drop some columns for now 

In [20]:
Y_train = X["outcome"]
X_train = X.drop(['people_id', 'activity_id', 'date_x', 'outcome', 'date_y', 'char_10_x', "group_1"], 1)

In [21]:
X_train.head()

,activity_category,char_1_x,char_2_x,char_3_x,char_4_x,char_5_x,char_6_x,char_7_x,char_8_x,char_9_x,...,char_29,char_30,char_31,char_32,char_33,char_34,char_35,char_36,char_37,char_38
0,type 4,type 2,type 2,type 1,type 3,type 6,type 2,type 1,type 4,type 8,...,0,1,1,0,0,1,1,1,0,36
1,type 2,type 2,type 2,type 1,type 3,type 6,type 2,type 1,type 4,type 8,...,0,1,1,0,0,1,1,1,0,36
2,type 2,type 2,type 2,type 1,type 3,type 6,type 2,type 1,type 4,type 8,...,0,1,1,0,0,1,1,1,0,36
3,type 2,type 2,type 2,type 1,type 3,type 6,type 2,type 1,type 4,type 8,...,0,1,1,0,0,1,1,1,0,36
4,type 2,type 2,type 2,type 1,type 3,type 6,type 2,type 1,type 4,type 8,...,0,1,1,0,0,1,1,1,0,36


# Convert the feature into binary features using dummpy variables

In [22]:
for c in X_train:
    if X_train[c].dtype == np.dtype('O') or X_train[c].dtype == np.dtype('bool'):
        print "Working with column " + str(c) 
        dummies = pd.get_dummies(X_train[c], prefix=c)
        X_train[dummies.columns] = dummies 
        X_train = X_train.drop([c], 1)

Working with column activity_category
Working with column char_1_x
Working with column char_2_x
Working with column char_3_x
Working with column char_4_x
Working with column char_5_x
Working with column char_6_x
Working with column char_7_x
Working with column char_8_x
Working with column char_9_x
Working with column char_1_y
Working with column char_2_y
Working with column char_3_y
Working with column char_4_y
Working with column char_5_y
Working with column char_6_y
Working with column char_7_y
Working with column char_8_y
Working with column char_9_y


In [23]:
X_train.head()

,char_10_y,char_11,char_12,char_13,char_14,char_15,char_16,char_17,char_18,char_19,...,char_8_y_type 8,char_9_y_type 1,char_9_y_type 2,char_9_y_type 3,char_9_y_type 4,char_9_y_type 5,char_9_y_type 6,char_9_y_type 7,char_9_y_type 8,char_9_y_type 9
0,1,0,0,1,1,0,1,0,0,0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,0,0,1,1,0,1,0,0,0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,0,0,1,1,0,1,0,0,0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,0,0,1,1,0,1,0,0,0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,0,0,1,1,0,1,0,0,0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Rescaling Features to a same range

In [24]:
from sklearn.preprocessing import MinMaxScaler
mms = MinMaxScaler()
X_train_norm = mms.fit_transform(X_train)

In [25]:
X_train_norm

array([[ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 1.,  1.,  1., ...,  0.,  0.,  0.],
       [ 1.,  1.,  1., ...,  0.,  0.,  0.],
       [ 1.,  1.,  1., ...,  0.,  0.,  0.]])

# Dimensionality Reduction PCA

In [26]:
from sklearn.decomposition import PCA
pca = PCA(n_components=140)
pca.fit(X_train_norm)
print(sum(pca.explained_variance_ratio_))

0.990970307364


In [27]:
X_train_norm_pca = pca.transform(X_train_norm)

# Stratified K fold cross validation - Random Forrest Classifier

In [136]:
from sklearn.cross_validation import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier  
from sklearn.metrics import accuracy_score

y_ = Y_train.values
X_ = X_train_norm_pca

rf_best_model = None
max_score = 0.0

skf = StratifiedKFold(y_, 5)

estimators = [5, 10, 20, 30]

for est in estimators:
    accuracy = 0.0 
    for train, test in skf:
        clf = RandomForestClassifier(n_estimators=est, random_state=0, n_jobs=4)
        clf.fit(X_[train], y_[train])
        pred = clf.predict(X_[test])
        accuracy += accuracy_score(y_[test], pred)
    avg_score = accuracy/5.0
    if avg_score > max_score:
        max_score = avg_score
        rf_best_model = clf
    print "Estimator value est : " + str(est) + " mean accuracy is " + str(accuracy/5.0)

Estimator value est : 5 mean accuracy is 0.826322497572
Estimator value est : 10 mean accuracy is 0.81197891474
Estimator value est : 20 mean accuracy is 0.8225469296
Estimator value est : 30 mean accuracy is 0.824551214955


Random Forrest best cross validation accuracy so far :- 0.826322497572

# Stratified K Fold Cross Validation Logistic Regression

In [ ]:
from sklearn.cross_validation import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier  
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression

y_ = Y_train.values
X_ = X_train_norm_pca

lr_best_model = None
max_score = 0.0

skf = StratifiedKFold(y_, 5)

Cv = [0.001, 0.01, 0.1, 1.0, 5.0]

for c in Cv:
    accuracy = 0.0 
    for train, test in skf:
        clf = LogisticRegression(C=c, penalty="l2", random_state=0, n_jobs=5)
        clf.fit(X_[train], y_[train])
        pred = clf.predict(X_[test])
        accuracy += accuracy_score(y_[test], pred)
    avg_score = accuracy/5.0
    if avg_score > max_score:
        max_score = avg_score
        lr_best_model = clf
    print "Estimator value c : " + str(c) + " mean accuracy is " + str(accuracy/5.0)

Estimator value c : 0.001 mean accuracy is 0.8347114611
Estimator value c : 0.01 mean accuracy is 0.834581301061
Estimator value c : 0.1 mean accuracy is 0.834691892185


Logistic Regression best cross validation accuracy so far :- 0.835335412088

In [33]:
def modelfit(alg, dtrain, labels, performCV=True, printFeatureImportance=True, cv_folds=5):
    #Fit the algorithm on the data
    alg.fit(dtrain, labels)
        
    #Predict training set:
    dtrain_predictions = alg.predict(dtrain)
    dtrain_predprob = alg.predict_proba(dtrain)[:,1]
    
    #Perform cross-validation:
    if performCV:
        cv_score = cross_validation.cross_val_score(alg, dtrain, labels, cv=cv_folds, scoring='roc_auc')
    
    #Print model report:
    print "\nModel Report"
    print "Accuracy : %.4g" % metrics.accuracy_score(labels.values, dtrain_predictions)
    print "AUC Score (Train): %f" % metrics.roc_auc_score(labels, dtrain_predprob)
    
    if performCV:
        print "CV Score : Mean - %.7g | Std - %.7g | Min - %.7g | Max - %.7g" % (np.mean(cv_score),np.std(cv_score),np.min(cv_score),np.max(cv_score))
        
    #Print Feature Importance:
    #if printFeatureImportance:
    #    feat_imp = pd.Series(alg.feature_importances_, predictors).sort_values(ascending=False)
    #    feat_imp.plot(kind='bar', title='Feature Importances')
    #    plt.ylabel('Feature Importance Score')

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
gbm0 = GradientBoostingClassifier(random_state=10)
modelfit(gbm0, X_train_norm_pca, Y_train.values)

# Prediction on test set

In [143]:
X_test = pd.merge(df_test, person, on='people_id')
X_test = DataFrameImputer().fit(X).transform(X_test)

In [144]:
# Convert booleans to integers.
p_logi = X_test.select_dtypes(include=['bool']).columns
X_test[p_logi] = X_test[p_logi].astype('int')
del p_logi

X_test['date_x'] = pd.to_datetime(X_test['date_x'])
X_test['date_y'] = pd.to_datetime(X_test['date_y'])
X_test.head()

,people_id,activity_id,date_x,activity_category,char_1_x,char_2_x,char_3_x,char_4_x,char_5_x,char_6_x,...,char_29,char_30,char_31,char_32,char_33,char_34,char_35,char_36,char_37,char_38
0,ppl_100004,act1_249281,2022-07-20,type 1,type 5,type 10,type 5,type 1,type 6,type 1,...,True,True,True,True,True,True,True,True,True,76
1,ppl_100004,act2_230855,2022-07-20,type 5,type 2,type 2,type 1,type 3,type 6,type 2,...,True,True,True,True,True,True,True,True,True,76
2,ppl_10001,act1_240724,2022-10-14,type 1,type 12,type 1,type 5,type 4,type 6,type 1,...,False,True,True,True,True,True,True,True,True,90
3,ppl_10001,act1_83552,2022-11-27,type 1,type 20,type 10,type 5,type 4,type 6,type 1,...,False,True,True,True,True,True,True,True,True,90
4,ppl_10001,act2_1043301,2022-10-15,type 5,type 2,type 2,type 1,type 3,type 6,type 2,...,False,True,True,True,True,True,True,True,True,90


In [145]:
X_test = X_test.drop(['people_id', 'activity_id', 'date_x', 'date_y', 'char_10_x', "group_1"], 1)

In [146]:
for c in X_test:
    if X_test[c].dtype == np.dtype('O') or X_test[c].dtype == np.dtype('bool'):
        dummies = pd.get_dummies(X_test[c], prefix=c)
        X_test[dummies.columns] = dummies 
        X_test = X_test.drop([c], 1)

In [147]:
# get the columns in train that are not in test
col_to_add = np.setdiff1d(X_train.columns, X_test.columns)

In [148]:
# add these columns to test, setting them equal to zero
for c in col_to_add:
    X_test[c] = 0

# select and reorder the test columns using the train columns
X_test = X_test[X_train.columns]

In [149]:
X_test.head()

,char_38,activity_category_type 1,activity_category_type 2,activity_category_type 3,activity_category_type 4,activity_category_type 5,activity_category_type 6,activity_category_type 7,char_1_x_type 1,char_1_x_type 10,...,char_33_False,char_33_True,char_34_False,char_34_True,char_35_False,char_35_True,char_36_False,char_36_True,char_37_False,char_37_True
0,76,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0
1,76,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0
2,90,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0
3,90,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0
4,90,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0


In [150]:
X_test_norm = mms.transform(X_test)

In [151]:
X_test_norm

array([[ 0.76,  1.  ,  0.  , ...,  1.  ,  0.  ,  1.  ],
       [ 0.76,  0.  ,  0.  , ...,  1.  ,  0.  ,  1.  ],
       [ 0.9 ,  1.  ,  0.  , ...,  1.  ,  0.  ,  1.  ],
       ..., 
       [ 0.36,  0.  ,  1.  , ...,  0.  ,  1.  ,  0.  ],
       [ 0.36,  0.  ,  1.  , ...,  0.  ,  1.  ,  0.  ],
       [ 0.36,  0.  ,  1.  , ...,  0.  ,  1.  ,  0.  ]])

In [153]:
X_test_norm_pca = pca.transform(X_test_norm)

# Random Forrest 

In [154]:
preds_test = rf_best_model.predict(X_test_norm_pca)

In [155]:
preds_test

array([1, 1, 1, ..., 0, 0, 0])

In [156]:
sub = pd.DataFrame()
sub['activity_id'] = df_test['activity_id']
sub['outcome'] = preds_test
sub.to_csv('random_forest_submission.csv', index=False)

# Logistic Regression 

In [164]:
preds_test = lr_best_model.predict(X_test_norm_pca)
sub = pd.DataFrame()
sub['activity_id'] = df_test['activity_id']
sub['outcome'] = preds_test
sub.to_csv('logistic_regression_submission.csv', index=False)